In [1]:
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd

import os
import acquire as a
import wrangle as w

In [2]:
# df = pd.DataFrame(a.scrape_github_data())

In [3]:
# df

In [4]:
def acquire_github_data():
    if os.path.isfile('repos.csv'):
        return pd.read_csv('repos.csv', index_col=0)
    else:
        df = pd.DataFrame(a.scrape_github_data())
        df.to_csv('repos.csv')
        return df

In [5]:
df = w.acquire_github_data()

In [6]:
df

,repo,language,readme_contents
0,./WebGoat/WebGoat,JavaScript,# WebGoat 8: A deliberately insecure Web Appli...
1,./vercel/next-react-server-components,JavaScript,# Next.js App Router + React Server Components...
2,./LaurentMazare/tch-rs,Rust,# tch-rs\nRust bindings for the C++ api of PyT...
3,./ultralytics/yolov5,Python,"<div align=""center"">\n <p>\n <a align=""cen..."
4,./FastForwardTeam/FastForward,JavaScript,"<div align=""center"">\n<img src=""https://avatar..."
...,...,...,...
159,./deepinsight/insightface,Python,\n# InsightFace: 2D and 3D Face Analysis Proje...
160,./DominikDoom/a1111-sd-webui-tagcomplete,JavaScript,![tag_autocomplete_light](https://user-images....
161,./zloirock/core-js,JavaScript,![logo](https://user-images.githubusercontent....
162,./ChanseyIsTheBest/NX-60FPS-RES-GFX-Cheats,Python,# NX-60FPS-RES-GFX-Cheats\n\n![fpsdemo1](https...


In [7]:
df = w.prepare_github_df()

In [8]:
# There are no null values in the dataframe

df.isna().sum()

repo          0
language      0
original      0
clean         0
stemmed       0
lemmatized    0
dtype: int64

In [9]:
df.head(3)

,repo,language,original,clean,stemmed,lemmatized
0,./WebGoat/WebGoat,JavaScript,# WebGoat 8: A deliberately insecure Web Appli...,webgoat 8 deliberately insecure web applicatio...,webgoat 8 deliber insecur web applic buildhttp...,webgoat 8 deliber insecur web applic buildhttp...
1,./vercel/next-react-server-components,JavaScript,# Next.js App Router + React Server Components...,nextjs app router react server components try ...,nextj app router react server compon tri demo ...,nextj app router react server compon tri demo ...
2,./LaurentMazare/tch-rs,Rust,# tch-rs\nRust bindings for the C++ api of PyT...,tchrs rust bindings c api pytorch goal tch cra...,tchr rust bind c api pytorch goal tch crate pr...,tchr rust bind c api pytorch goal tch crate pr...


In [10]:
df.language.value_counts()

Python        58
JavaScript    56
Rust          50
Name: language, dtype: int64

In [11]:
# Splitting the data

from sklearn.model_selection import train_test_split

In [17]:
def split_data():
    df = w.prepare_github_df()
    
    train_validate, test = train_test_split(df, random_state = 1349, train_size=.8, stratify=df.language)

    train, validate = train_test_split(train_validate, random_state = 1349, train_size=.7, stratify=train_validate.language)
    
    return train, validate, test

In [13]:
train, validate, test = split_data()

In [14]:
train.shape, validate.shape, test.shape

((91, 6), (40, 6), (33, 6))

In [ ]:
# Prepare for modeling

In [18]:
def modeling_prep():
    
    train, validate, test = split_data()
    X_train = train['lemmatized']
    X_validate = validate['lemmatized']
    X_test = test['lemmatized']
    y_train = train['language']
    y_validate = validate['language']
    y_test = test['language']
    return X_train, X_validate, X_test, y_train, y_validate, y_test

In [19]:
X_train, X_validate, X_test, y_train, y_validate, y_test = modeling_prep()